In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Earthquake\ Shelter\ Project

Mounted at /content/drive
/content/drive/MyDrive/Earthquake Shelter Project


In [ ]:
!pip install osmnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 1.1 MB/s eta 0:00:00


In [ ]:
!pip install "pandas<2.0.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 48.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 1.5.3 which is incompatible.


In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import pickle
from shapely.geometry import Point, Polygon
from scipy.spatial.distance import cdist
from scipy.spatial import Delaunay, ConvexHull
import matplotlib.pyplot as plt
import networkx as nx
from collections import Counter
import folium
import osmnx as ox

In [ ]:
def read_pickle(dataset_name):
  with open('data/' + dataset_name + '.pkl', 'rb') as file_:
    return pickle.load(file_)

In [ ]:
def read_dbf(filepath):
  return gpd.read_file(filepath)

In [ ]:
PREF = 'chiba'
drop_small_widths = False

```python
public_facilities = read_dbf('public_locs.dbf')
public_facilities = public_facilities.rename(columns={
    'P02_001': 'admin_code',
    'P02_002': 'type_general',
    'P02_003': 'type_specific',
    'P02_004': 'name',
    'P02_005': 'address',
    'P02_006': 'maintenance_code'
}).drop(['P02_007'], axis=1)
```

```python
numerical_cols = ['admin_code', 'type_general', 'maintenance_code']
public_facilities[numerical_cols] = public_facilities[numerical_cols].apply(pd.to_numeric)
public_facilities
```

In [ ]:
def init_map(lat=35.6811939, lon=139.7654723):
  m = folium.Map(location=(lat, lon), zoom_start=15)
  return m

In [ ]:
def generate_map(map, latitudes, longitudes, color='blue'):
  m = map
  coordinates = list(zip(latitudes, longitudes))

  for coord in coordinates:
    folium.CircleMarker(location=coord, radius=2, color=color, fill=True, fill_color='blue').add_to(m)

  return m

In [ ]:
facilities = read_pickle(f'prefectural_data/{PREF}/facilities')
facilities

,admin_code,type_general,type_specific,name,address,maintenance_code,geometry,latitude,longitude
0,12101,3,03001,千葉県立美術館,中央港1-10-1,2,None,35.601220,140.101146
1,12101,3,03001,千葉市美術館,中央3-10-8,3,None,35.608958,140.124638
2,12204,3,03001,ふなばしアンゼルセン公園子ども美術館,金堀町525,3,None,35.758910,140.064302
3,12210,3,03001,茂原市立美術館・郷土資料館,高師1345-1,3,None,35.430138,140.280233
4,12211,3,03001,成田山書道美術館,成田640,4,None,35.787491,140.321857
...,...,...,...,...,...,...,...,...,...
7009,12441,19,19013,大多喜町立大多喜保育園,大多喜120,3,None,NaN,NaN
7010,12443,19,19013,御宿町立岩和田保育所,岩和田926-1,3,None,NaN,NaN
7011,12443,19,19013,御宿町立御宿保育所,久保2180-1,3,None,NaN,NaN
7012,12463,19,19013,鋸南町立勝山保育所,竜島973-4,3,None,35.115511,139.828551


In [ ]:
tokyo_boundary_polygon = ox.geocode_to_gdf(f'{PREF.capitalize()}, Japan').unary_union

In [ ]:
def is_inside_tokyo(latitude, longitude):
  point = Point(longitude, latitude)
  return point.within(tokyo_boundary_polygon)

In [ ]:
known_facilities = facilities[facilities['latitude'].notna()]
known_facilities

,admin_code,type_general,type_specific,name,address,maintenance_code,geometry,latitude,longitude
0,12101,3,03001,千葉県立美術館,中央港1-10-1,2,None,35.601220,140.101146
1,12101,3,03001,千葉市美術館,中央3-10-8,3,None,35.608958,140.124638
2,12204,3,03001,ふなばしアンゼルセン公園子ども美術館,金堀町525,3,None,35.758910,140.064302
3,12210,3,03001,茂原市立美術館・郷土資料館,高師1345-1,3,None,35.430138,140.280233
4,12211,3,03001,成田山書道美術館,成田640,4,None,35.787491,140.321857
...,...,...,...,...,...,...,...,...,...
7006,12441,19,19013,つぐみの森保育園,中野260,3,None,35.251571,140.201762
7007,12441,19,19013,大多喜町立上瀑保育園,横山3060,3,None,35.300017,140.250373
7008,12441,19,19013,大多喜町立総元保育園,大戸591,3,None,35.258549,140.246153
7012,12463,19,19013,鋸南町立勝山保育所,竜島973-4,3,None,35.115511,139.828551


In [ ]:
facilities_copy = known_facilities

tokyo_boundary_polygon = ox.geocode_to_gdf(f'{PREF.capitalize()}, Japan').unary_union

# Function to check if a point is inside Tokyo and on the mainland portion
def is_inside_tokyo_mainland(latitude, longitude):
    point = Point(longitude, latitude)
    return point.within(tokyo_boundary_polygon) and latitude > 35.49

# Filter out rows based on whether they are inside Tokyo and on the mainland portion
facilities_copy['inside_tokyo_mainland'] = facilities_copy.apply(lambda row: is_inside_tokyo_mainland(row['latitude'], row['longitude']), axis=1)

# Create a Folium map
m = folium.Map(location=[35.6895, 139.6917], zoom_start=10)

# Add circle markers for points inside Tokyo and on the mainland portion
for _, row in facilities_copy[facilities_copy['inside_tokyo_mainland']].iterrows():
    folium.CircleMarker(location=[row['latitude'], row['longitude']], radius=5, color='green', fill=True).add_to(m)

# Display the map
m

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# shouldn't work for chiba, kanagawa, etc.
# known_facilities = facilities_copy[facilities_copy['inside_tokyo_mainland']]

In [ ]:
m1 = init_map()
m1 = generate_map(m1, known_facilities['latitude'], known_facilities['longitude'])
m1

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
land_worth = read_pickle(f'prefectural_data/{PREF}/land_worth')
land_worth = land_worth[land_worth['latitude'].notna()]
land_worth

,monetary_worth,admin_code,ward,full_address,resident_address,area,building_type,stories_above_ground,stories_below_ground,road_width,nearest_station,dist_to_station,steel_frame,reinforced_concrete,lightweight,block,wood,latitude,longitude
0,300000,12101,千葉中央,"[千葉県, 千葉中央, 千葉市中央区新千葉, 3-320]","[新千葉, 3-4-16]",320,W,2,0,4.0,千葉,550,0,0,0,0,1,35.607368,140.117818
1,107000,12101,千葉中央,"[千葉県, 千葉中央, 千葉市中央区道場南, 2-190-13]","[道場南, 2-9-6]",138,LS,2,0,4.0,千葉,1900,1,0,1,0,0,35.607368,140.117818
2,93800,12101,千葉中央,"[千葉県, 千葉中央, 千葉市中央区矢作町, 942-25]",None,202,LS,2,0,4.0,千葉,3200,1,0,1,0,0,35.607368,140.117818
3,131000,12101,千葉中央,"[千葉県, 千葉中央, 千葉市中央区末広, 4-6-9]","[末広, 4-6-9]",165,W,2,0,6.0,千葉寺,600,0,0,0,0,1,35.607368,140.117818
4,273000,12101,千葉中央,"[千葉県, 千葉中央, 千葉市中央区松波, 1-19-5]","[松波, 1-18-14]",186,W,2,0,6.0,西千葉,650,0,0,0,0,1,35.607368,140.117818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,10800,12427,長南,"[千葉県, 長南町, 長生郡豊原字上ノ原, 2154-9]",None,251,W,2,0,5.0,茂原,7200,0,0,0,0,1,35.386036,140.237276
1255,12300,12427,長南,"[千葉県, 長南町, 長生郡長南字元宿, 1287-1]",None,328,LS,2,0,8.5,茂原,7900,1,0,1,0,0,35.386036,140.237276
1256,13300,12427,長南,"[千葉県, 長南町, 長生郡長南字土手町, 2546-1]",None,340,W,2,0,9.0,茂原,8600,0,0,0,0,1,35.386036,140.237276
1257,19800,12443,御宿,"[千葉県, 御宿町, 夷隅郡久保字南側, 2118]",None,532,W,1,0,3.6,御宿,620,0,0,0,0,1,35.191661,140.348777


In [ ]:
m_plots = generate_map(m1, land_worth['latitude'], land_worth['longitude'], color='red')
m_plots

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
def find_closest_plot(row_a, df_b, col_list):
  distances = cdist([[row_a['latitude'], row_a['longitude']]], df_b[['latitude', 'longitude']], 'euclidean')

  closest_index = np.argmin(distances)
  closest_plot = df_b.iloc[closest_index]

  for col in col_list:
    row_a[col] = closest_plot[col]

  return row_a

In [ ]:
col_list = ['area', 'stories_above_ground', 'stories_below_ground', 'road_width', 'nearest_station', 'dist_to_station',
            'steel_frame', 'reinforced_concrete', 'lightweight', 'block', 'wood']

In [ ]:
land_worth[['latitude', 'longitude']]

,latitude,longitude
0,35.607368,140.117818
1,35.607368,140.117818
2,35.607368,140.117818
3,35.607368,140.117818
4,35.607368,140.117818
...,...,...
1254,35.386036,140.237276
1255,35.386036,140.237276
1256,35.386036,140.237276
1257,35.191661,140.348777


In [ ]:
for col in col_list:
  known_facilities.loc[:, col] = 0

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [ ]:
for idx, shelter_row in known_facilities.iterrows():
  known_facilities.loc[idx] = find_closest_plot(shelter_row, land_worth, col_list)

In [ ]:
known_facilities

,admin_code,type_general,type_specific,name,address,maintenance_code,geometry,latitude,longitude,inside_tokyo_mainland,...,stories_above_ground,stories_below_ground,road_width,nearest_station,dist_to_station,steel_frame,reinforced_concrete,lightweight,block,wood
0,12101,3,03001,千葉県立美術館,中央港1-10-1,2,None,35.601220,140.101146,True,...,2,0,5.0,千葉みなと,1200,0,0,0,0,1
1,12101,3,03001,千葉市美術館,中央3-10-8,3,None,35.608958,140.124638,True,...,2,0,4.0,千葉,550,0,0,0,0,1
2,12204,3,03001,ふなばしアンゼルセン公園子ども美術館,金堀町525,3,None,35.758910,140.064302,True,...,2,0,5.5,三咲,3200,0,0,0,0,1
3,12210,3,03001,茂原市立美術館・郷土資料館,高師1345-1,3,None,35.430138,140.280233,False,...,2,0,4.0,新茂原,1200,1,0,1,0,0
4,12211,3,03001,成田山書道美術館,成田640,4,None,35.787491,140.321857,True,...,2,0,4.5,京成成田,1000,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7006,12441,19,19013,つぐみの森保育園,中野260,3,None,35.251571,140.201762,False,...,2,0,5.0,茂原,7000,0,0,0,0,1
7007,12441,19,19013,大多喜町立上瀑保育園,横山3060,3,None,35.300017,140.250373,False,...,2,0,5.0,茂原,7000,0,0,0,0,1
7008,12441,19,19013,大多喜町立総元保育園,大戸591,3,None,35.258549,140.246153,False,...,1,0,3.6,御宿,620,0,0,0,0,1
7012,12463,19,19013,鋸南町立勝山保育所,竜島973-4,3,None,35.115511,139.828551,False,...,2,0,3.8,館山,650,0,0,0,0,1


In [ ]:
if drop_small_widths:
  known_facilities = known_facilities[known_facilities['road_width'] > 12.0]
known_facilities

,admin_code,type_general,type_specific,name,address,maintenance_code,geometry,latitude,longitude,inside_tokyo_mainland,...,stories_above_ground,stories_below_ground,road_width,nearest_station,dist_to_station,steel_frame,reinforced_concrete,lightweight,block,wood
0,12101,3,03001,千葉県立美術館,中央港1-10-1,2,None,35.601220,140.101146,True,...,2,0,5.0,千葉みなと,1200,0,0,0,0,1
1,12101,3,03001,千葉市美術館,中央3-10-8,3,None,35.608958,140.124638,True,...,2,0,4.0,千葉,550,0,0,0,0,1
2,12204,3,03001,ふなばしアンゼルセン公園子ども美術館,金堀町525,3,None,35.758910,140.064302,True,...,2,0,5.5,三咲,3200,0,0,0,0,1
3,12210,3,03001,茂原市立美術館・郷土資料館,高師1345-1,3,None,35.430138,140.280233,False,...,2,0,4.0,新茂原,1200,1,0,1,0,0
4,12211,3,03001,成田山書道美術館,成田640,4,None,35.787491,140.321857,True,...,2,0,4.5,京成成田,1000,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7006,12441,19,19013,つぐみの森保育園,中野260,3,None,35.251571,140.201762,False,...,2,0,5.0,茂原,7000,0,0,0,0,1
7007,12441,19,19013,大多喜町立上瀑保育園,横山3060,3,None,35.300017,140.250373,False,...,2,0,5.0,茂原,7000,0,0,0,0,1
7008,12441,19,19013,大多喜町立総元保育園,大戸591,3,None,35.258549,140.246153,False,...,1,0,3.6,御宿,620,0,0,0,0,1
7012,12463,19,19013,鋸南町立勝山保育所,竜島973-4,3,None,35.115511,139.828551,False,...,2,0,3.8,館山,650,0,0,0,0,1


In [ ]:
shelters = read_pickle('shelters')
tokyo_shelters = shelters[shelters['prefecture'] == PREF]
tokyo_shelters

,municality_code,name,address,type,capacity,area,earthquake,tsunami,water_damage,volcano,other,not_specified,level,notes,latitude,longitude,num,geometry,prefecture
3919,12101,ＪＦＥ健保千葉グランド一帯,千葉県千葉市中央区南町1丁目,広域避難所,-1,-1,False,False,False,False,False,True,3,None,35.585171,140.130926,1,None,chiba
3920,12101,そが野ふれあい公園,千葉県千葉市中央区蘇我2,避難場所、避難所、避難施設,4400,-1,False,False,False,False,False,True,1,None,35.570734,140.134888,2,None,chiba
3921,12101,ニューサンピア千葉一帯,千葉県千葉市中央区仁戸名町705,広域避難所,-1,-1,False,False,False,False,False,True,3,None,35.578529,140.160787,3,None,chiba
3922,12101,ビジネス支援センター,千葉県千葉市中央区中央4-5-1,避難場所、避難所、避難施設,60,-1,False,False,False,False,False,True,3,None,35.607447,140.123346,4,None,chiba
3923,12101,みなと公園,千葉県千葉市中央区千葉港6,広域避難所,-1,-1,False,False,False,False,False,True,1,None,35.606581,140.104835,5,None,chiba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7385,12234,小川公会堂,千葉県南房総市,一時避難場所,-1,-1,False,False,False,False,False,True,3,None,35.042728,139.997498,4329,None,chiba
7386,12234,中三原区集会所,千葉県南房総市,一時避難場所,-1,-1,False,False,False,False,False,True,3,None,35.041222,139.990214,4330,None,chiba
7387,12234,峰金谷集会所,千葉県南房総市,一時避難場所,-1,-1,False,False,False,False,False,True,3,None,35.031485,139.987051,4331,None,chiba
7388,12234,龍神社社務所,千葉県南房総市,一時避難場所,-1,-1,False,False,False,False,False,True,3,None,35.027881,139.992071,4332,None,chiba


In [ ]:
m2 = generate_map(m1, tokyo_shelters['latitude'], tokyo_shelters['longitude'], color='green')
m2

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
shelters_in_common = known_facilities.merge(tokyo_shelters, on='name', how='inner')
shelters_in_common.drop_duplicates(subset=['name'], keep='first', inplace=True, ignore_index=True)
shelters_in_common

,admin_code,type_general,type_specific,name,address_x,maintenance_code,geometry_x,latitude_x,longitude_x,inside_tokyo_mainland,...,volcano,other,not_specified,level,notes,latitude_y,longitude_y,num,geometry_y,prefecture
0,12217,11,11223,気象大学校,旭町7-4-81,1,None,35.859861,139.958432,True,...,False,False,True,1,None,35.859052,139.957549,1459,None,chiba
1,12215,12,12002,旭市役所,ニ1920,3,None,35.720435,140.646733,True,...,False,False,True,1,None,35.720391,140.646596,1340,None,chiba
2,12230,12,12002,八街市役所,八街ほ35-29,3,None,35.665759,140.318166,True,...,False,False,True,1,None,35.665859,140.317913,2347,None,chiba
3,12232,12,12002,白井市役所,復1123,3,None,35.791546,140.056335,True,...,False,False,True,1,None,35.791503,140.056372,2416,None,chiba
4,12238,12,12002,いすみ市役所,大原7400-1,3,None,35.253993,140.384843,False,...,False,False,True,1,None,35.253887,140.385117,2837,None,chiba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,12410,19,19013,光町保育園,宮川11796-2,4,None,35.665756,140.506920,True,...,False,False,False,1,None,35.665756,140.506920,3139,None,chiba
59,12423,19,19013,一松保育所,一松丙3718-2,3,None,35.403616,140.370305,False,...,False,False,True,1,None,35.403616,140.370305,3281,None,chiba
60,12423,19,19013,高根保育所,本郷6937,3,None,35.415364,140.361111,False,...,False,False,True,1,None,35.415364,140.361111,3284,None,chiba
61,12423,19,19013,八積保育所,金田2727,3,None,35.395172,140.342725,False,...,False,False,True,1,None,35.395172,140.342725,3289,None,chiba


In [ ]:
len(list(set(shelters_in_common['name'])))

63

In [ ]:
threshold_distance = 0.05  # km

distances = cdist(tokyo_shelters[['latitude', 'longitude']], known_facilities[['latitude', 'longitude']], 'euclidean')

# Find the minimum distance and its position
min_distance = np.min(distances)
min_indices = np.where(distances == min_distance)

# Check if distance is less than threshold and store matching indices
matching_indices = list(zip(min_indices[0], min_indices[1])) if min_distance < threshold_distance else []

In [ ]:
repeated_indices = [idx for idx, count in Counter(matching_indices).items() if count > 1]

In [ ]:
len(matching_indices)

168

In [ ]:
print(shelters_in_common['name'])

0        気象大学校
1         旭市役所
2        八街市役所
3        白井市役所
4       いすみ市役所
        ...   
58       光町保育園
59       一松保育所
60       高根保育所
61       八積保育所
62    つぐみの森保育園
Name: name, Length: 63, dtype: object


In [ ]:
known_facilities.loc[:, 'shelter'] = 0
known_facilities_matches = [sec for _, sec in matching_indices]

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [ ]:
max(known_facilities_matches)

5062

In [ ]:
known_facilities_matching_indices = [known_facilities.index[idx] for idx in known_facilities_matches]
len(known_facilities_matching_indices)

168

In [ ]:
for idx, row in known_facilities.iterrows():
  if row['name'] in shelters_in_common['name'] or idx in known_facilities_matching_indices:
    known_facilities.loc[idx, 'shelter'] = 1

In [ ]:
mainland_known_facilities = known_facilities[known_facilities['latitude'] > 35.4]
mainland_known_facilities = mainland_known_facilities.reset_index()
mainland_known_facilities

,index,admin_code,type_general,type_specific,name,address,maintenance_code,geometry,latitude,longitude,...,stories_below_ground,road_width,nearest_station,dist_to_station,steel_frame,reinforced_concrete,lightweight,block,wood,shelter
0,0,12101,3,03001,千葉県立美術館,中央港1-10-1,2,None,35.601220,140.101146,...,0,5.0,千葉みなと,1200,0,0,0,0,1,0
1,1,12101,3,03001,千葉市美術館,中央3-10-8,3,None,35.608958,140.124638,...,0,4.0,千葉,550,0,0,0,0,1,0
2,2,12204,3,03001,ふなばしアンゼルセン公園子ども美術館,金堀町525,3,None,35.758910,140.064302,...,0,5.5,三咲,3200,0,0,0,0,1,0
3,3,12210,3,03001,茂原市立美術館・郷土資料館,高師1345-1,3,None,35.430138,140.280233,...,0,4.0,新茂原,1200,1,0,1,0,0,0
4,4,12211,3,03001,成田山書道美術館,成田640,4,None,35.787491,140.321857,...,0,4.5,京成成田,1000,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4200,6998,12423,19,19013,高根保育所,本郷6937,3,None,35.415364,140.361111,...,0,5.0,上総一ノ宮,2300,0,0,0,0,1,1
4201,7000,12424,19,19013,関保育所,関6722,3,None,35.445861,140.358195,...,0,4.0,八積,9100,0,0,0,0,1,0
4202,7001,12424,19,19013,南白亀保育所,牛込85,3,None,35.466552,140.384579,...,0,4.0,八積,9100,0,0,0,0,1,0
4203,7003,12426,19,19013,三島野保育所,六地蔵344-2,3,None,35.452466,140.215209,...,0,5.0,茂原,7000,0,0,0,0,1,0


In [ ]:
mk = init_map()
generate_map(mk, mainland_known_facilities['latitude'], mainland_known_facilities['longitude'])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
mainland_known_facilities.columns

Index(['index', 'admin_code', 'type_general', 'type_specific', 'name',
       'address', 'maintenance_code', 'geometry', 'latitude', 'longitude',
       'inside_tokyo_mainland', 'area', 'stories_above_ground',
       'stories_below_ground', 'road_width', 'nearest_station',
       'dist_to_station', 'steel_frame', 'reinforced_concrete', 'lightweight',
       'block', 'wood', 'shelter'],
      dtype='object')

In [ ]:
points = np.column_stack((mainland_known_facilities['latitude'], mainland_known_facilities['longitude']))

In [ ]:
points.shape

(4205, 2)

In [ ]:
tri = Delaunay(points)
indices = tri.simplices.flatten()

In [ ]:
adj_matrix = np.zeros((len(points), len(points)), dtype=int)

for i in range(0, len(indices), 3):
  simplex = indices[i:i+3]
  for i in range(3):
    for j in range(i+1, 3):
      adj_matrix[simplex[i], simplex[j]] = 1
      adj_matrix[simplex[j], simplex[i]] = 1

In [ ]:
adj_matrix.shape

(4205, 4205)

In [ ]:
row_sums = np.sum(adj_matrix, axis=1)
unconnected_indices = np.where(row_sums == 0)[0]
print(len(unconnected_indices))
mainland_known_facilities.loc[unconnected_indices, 'shelter'] == 1

194


56      False
215     False
227     False
268     False
272     False
        ...  
3730    False
3742    False
3754    False
3763    False
4198     True
Name: shelter, Length: 194, dtype: bool

In [ ]:
rows_to_keep = np.any(adj_matrix, axis=1)
cols_to_keep = np.any(adj_matrix, axis=0)

# Filter the adjacency matrix based on the rows and columns to keep
filtered_adj_matrix = adj_matrix[rows_to_keep][:, cols_to_keep]
filtered_adj_matrix.shape

(4011, 4011)

In [ ]:
facilities2 = mainland_known_facilities.drop(unconnected_indices)
facilities2

,index,admin_code,type_general,type_specific,name,address,maintenance_code,geometry,latitude,longitude,...,stories_below_ground,road_width,nearest_station,dist_to_station,steel_frame,reinforced_concrete,lightweight,block,wood,shelter
0,0,12101,3,03001,千葉県立美術館,中央港1-10-1,2,None,35.601220,140.101146,...,0,5.0,千葉みなと,1200,0,0,0,0,1,0
1,1,12101,3,03001,千葉市美術館,中央3-10-8,3,None,35.608958,140.124638,...,0,4.0,千葉,550,0,0,0,0,1,0
2,2,12204,3,03001,ふなばしアンゼルセン公園子ども美術館,金堀町525,3,None,35.758910,140.064302,...,0,5.5,三咲,3200,0,0,0,0,1,0
3,3,12210,3,03001,茂原市立美術館・郷土資料館,高師1345-1,3,None,35.430138,140.280233,...,0,4.0,新茂原,1200,1,0,1,0,0,0
4,4,12211,3,03001,成田山書道美術館,成田640,4,None,35.787491,140.321857,...,0,4.5,京成成田,1000,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4200,6998,12423,19,19013,高根保育所,本郷6937,3,None,35.415364,140.361111,...,0,5.0,上総一ノ宮,2300,0,0,0,0,1,1
4201,7000,12424,19,19013,関保育所,関6722,3,None,35.445861,140.358195,...,0,4.0,八積,9100,0,0,0,0,1,0
4202,7001,12424,19,19013,南白亀保育所,牛込85,3,None,35.466552,140.384579,...,0,4.0,八積,9100,0,0,0,0,1,0
4203,7003,12426,19,19013,三島野保育所,六地蔵344-2,3,None,35.452466,140.215209,...,0,5.0,茂原,7000,0,0,0,0,1,0


In [ ]:
facilities2[facilities2['shelter'] == 1]

,index,admin_code,type_general,type_specific,name,address,maintenance_code,geometry,latitude,longitude,...,stories_below_ground,road_width,nearest_station,dist_to_station,steel_frame,reinforced_concrete,lightweight,block,wood,shelter
40,57,12220,3,03002,流山市立博物館,加1-1225-6,3,None,35.858662,139.902498,...,0,5.0,平和台,200,0,0,0,0,1,1
201,262,12215,9,09001,日本郵政公社加入者福祉施設旭簡易保険保養センター,仁玉2280-1,0,None,35.688147,140.651961,...,0,5.4,旭,600,0,0,0,0,1,1
881,1198,12213,14,14006,千葉県警察学校,士農田28-1,2,None,35.575170,140.413150,...,0,4.0,成東,1800,0,0,0,0,1,1
1031,1578,12203,16,16001,市立信篤小学校,原木2-16-1,3,None,35.698017,139.943584,...,0,5.3,西船橋,1200,0,0,0,0,1,1
1032,1583,12203,16,16001,市立曽谷小学校,曽谷7-18-1,3,None,35.746899,139.926751,...,0,6.0,市川,3800,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4194,6985,12410,19,19013,横芝町立上堺保育所,新島3510-1,3,None,35.620539,140.507813,...,0,4.2,横芝,1800,0,0,0,0,1,1
4195,6986,12410,19,19013,横芝町立大総保育所,寺方760,3,None,35.677420,140.460589,...,0,4.2,横芝,1800,0,0,0,0,1,1
4197,6988,12410,19,19013,光町保育園,宮川11796-2,4,None,35.665756,140.506920,...,0,4.2,横芝,1800,0,0,0,0,1,1
4199,6997,12423,19,19013,一松保育所,一松丙3718-2,3,None,35.403616,140.370305,...,0,5.0,上総一ノ宮,2300,0,0,0,0,1,1


In [ ]:
m3 = init_map()
m3 = generate_map(m3, mainland_known_facilities['latitude'], mainland_known_facilities['longitude'], color='blue')
m3 = generate_map(m3, mainland_known_facilities.loc[unconnected_indices, 'latitude'], mainland_known_facilities.loc[unconnected_indices, 'longitude'], color='red')
m3

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
hull = ConvexHull(points)
len(hull.vertices)

11

In [ ]:
hull_points = [points[vertex] for vertex in hull.vertices]

# Compute center of the convex hull
center = np.mean(hull_points, axis=0)

# Create a Folium map centered at the center of the convex hull
m = folium.Map(location=center, zoom_start=10)

# Plot all points as circle markers
for point in points:
    folium.CircleMarker(location=point, radius=3, color='red', fill=True).add_to(m)

# Plot the convex hull as a polygon
folium.Polygon(locations=hull_points, color='blue', fill=True).add_to(m)

# Display the map
m

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
a = filtered_adj_matrix
a.shape

(4011, 4011)

In [ ]:
facilities2.columns

Index(['index', 'admin_code', 'type_general', 'type_specific', 'name',
       'address', 'maintenance_code', 'geometry', 'latitude', 'longitude',
       'inside_tokyo_mainland', 'area', 'stories_above_ground',
       'stories_below_ground', 'road_width', 'nearest_station',
       'dist_to_station', 'steel_frame', 'reinforced_concrete', 'lightweight',
       'block', 'wood', 'shelter'],
      dtype='object')

In [ ]:
X_cols = ['stories_above_ground', 'stories_below_ground', 'steel_frame', 'reinforced_concrete', 'lightweight', 'block', 'wood']
X_df = facilities2[X_cols]
X_df

,stories_above_ground,stories_below_ground,steel_frame,reinforced_concrete,lightweight,block,wood
0,2,0,0,0,0,0,1
1,2,0,0,0,0,0,1
2,2,0,0,0,0,0,1
3,2,0,1,0,1,0,0
4,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...
4200,2,0,0,0,0,0,1
4201,2,0,0,0,0,0,1
4202,2,0,0,0,0,0,1
4203,2,0,0,0,0,0,1


In [ ]:
with open('data/all_features.pkl', 'rb') as f:
  all_features = pickle.load(f)
all_features = all_features[all_features['prefecture'] == PREF]
all_features

,mesh_id,latitude,longitude,area,rice_paddy,other_agriculture,forest,wasteland,building,road_transport,...,shelter_capacity,admin_code,2015,total,0-14,15-64,65+,75+,80+,prefecture
14178,52392686,34.90000,139.8250,1055381,0.000000,0.090000,0.09,0.01,0.00,0.0,...,0,12234,12.0242,10.7435,0.8384,5.1512,4.7539,1.8092,1.8092,chiba
14179,52392687,34.90000,139.8375,1055374,0.060000,0.340002,0.06,0.02,0.34,0.0,...,0,12234,266.2291,212.0187,9.3231,82.7874,119.9082,85.5635,53.6240,chiba
14180,52392688,34.90000,139.8500,1055368,0.060000,0.429999,0.07,0.00,0.16,0.0,...,0,12234,26.0224,21.3664,1.4584,7.0723,12.8357,10.8912,5.6049,chiba
14181,52392689,34.90000,139.8625,1055362,0.050000,0.190000,0.09,0.04,0.25,0.0,...,0,12234,326.3613,263.1675,20.0778,77.3496,165.7401,118.2256,71.5858,chiba
14182,52392695,34.90833,139.8125,1055282,0.000000,0.000000,0.00,0.00,0.00,0.0,...,0,12205,37.0263,29.1104,2.0890,8.1765,18.8449,12.9438,8.6808,chiba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7775,54390693,36.07500,139.7875,1040339,0.670001,0.000000,0.00,0.00,0.32,0.0,...,0,12208,399.0770,374.0179,48.3071,189.1237,136.5871,64.4181,39.4870,chiba
8279,54391602,36.08333,139.7750,1040237,0.210000,0.020000,0.00,0.00,0.31,0.0,...,0,12208,509.5154,463.1355,45.3290,226.2061,191.6003,106.5626,62.2848,chiba
8280,54391603,36.08333,139.7875,1040230,0.470000,0.020000,0.00,0.00,0.37,0.0,...,0,12208,843.0849,805.0672,86.4377,471.2194,247.4101,126.7250,68.9844,chiba
8289,54391612,36.09167,139.7750,1040128,0.220000,0.010000,0.00,0.00,0.09,0.0,...,0,12208,199.1561,177.1850,10.5859,79.8944,86.7047,42.1453,20.5926,chiba


In [ ]:
all_features.columns

Index(['mesh_id', 'latitude', 'longitude', 'area', 'rice_paddy',
       'other_agriculture', 'forest', 'wasteland', 'building',
       'road_transport', 'rail_transport', 'other_site', 'river_wetland',
       'seashore', 'sea', 'golf_course', 'unknown', 'dist_to_water',
       'shelter_count', 'shelter_capacity', 'admin_code', '2015', 'total',
       '0-14', '15-64', '65+', '75+', '80+', 'prefecture'],
      dtype='object')

In [ ]:
lat_width = 75 / 9000
lon_height = 0.0125
bottom_lat = 35.5
bottom_lon = 139.4625
error = 0.001

In [ ]:
cols_to_add = ['rice_paddy',
       'other_agriculture', 'forest', 'wasteland', 'building',
       'road_transport', 'rail_transport', 'other_site', 'river_wetland',
       'seashore', 'sea', 'golf_course', 'unknown', 'dist_to_water',
       '0-14', '15-64', '65+', '75+', '80+']

In [ ]:
for col in cols_to_add:
  facilities2.loc[:, col] = None
facilities2

,index,admin_code,type_general,type_specific,name,address,maintenance_code,geometry,latitude,longitude,...,seashore,sea,golf_course,unknown,dist_to_water,0-14,15-64,65+,75+,80+
0,0,12101,3,03001,千葉県立美術館,中央港1-10-1,2,None,35.601220,140.101146,...,None,None,None,None,None,None,None,None,None,None
1,1,12101,3,03001,千葉市美術館,中央3-10-8,3,None,35.608958,140.124638,...,None,None,None,None,None,None,None,None,None,None
2,2,12204,3,03001,ふなばしアンゼルセン公園子ども美術館,金堀町525,3,None,35.758910,140.064302,...,None,None,None,None,None,None,None,None,None,None
3,3,12210,3,03001,茂原市立美術館・郷土資料館,高師1345-1,3,None,35.430138,140.280233,...,None,None,None,None,None,None,None,None,None,None
4,4,12211,3,03001,成田山書道美術館,成田640,4,None,35.787491,140.321857,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4200,6998,12423,19,19013,高根保育所,本郷6937,3,None,35.415364,140.361111,...,None,None,None,None,None,None,None,None,None,None
4201,7000,12424,19,19013,関保育所,関6722,3,None,35.445861,140.358195,...,None,None,None,None,None,None,None,None,None,None
4202,7001,12424,19,19013,南白亀保育所,牛込85,3,None,35.466552,140.384579,...,None,None,None,None,None,None,None,None,None,None
4203,7003,12426,19,19013,三島野保育所,六地蔵344-2,3,None,35.452466,140.215209,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
facilities_copy = facilities2

## Making the heatmap here

In [ ]:
all_features['known_shelter_count'] = 0
all_features[['latitude', 'longitude', 'shelter_count', 'known_shelter_count']]

,latitude,longitude,shelter_count,known_shelter_count
14178,34.90000,139.8250,0,0
14179,34.90000,139.8375,0,0
14180,34.90000,139.8500,0,0
14181,34.90000,139.8625,1,0
14182,34.90833,139.8125,0,0
...,...,...,...,...
7775,36.07500,139.7875,1,0
8279,36.08333,139.7750,2,0
8280,36.08333,139.7875,2,0
8289,36.09167,139.7750,0,0


In [ ]:
facilities_copy['shelter'].sum()

114

In [ ]:
facilities_copy.shape

(4011, 42)

In [ ]:
for idx, row in facilities_copy.iterrows():
  lat, lon = row['latitude'], row['longitude']
  latdisp, londisp = int((lat - bottom_lat) // lat_width), int((lon - bottom_lon) // lon_height)
  if latdisp < 0 or londisp < 0:
    continue
  lat_new, lon_new = bottom_lat + latdisp * lat_width, bottom_lon + londisp * lon_height

  for all_feat_idx, feature_row in all_features.iterrows():
    if abs(feature_row['latitude'] - lat_new) < error and abs(feature_row['longitude'] - lon_new) < error:
      for col in cols_to_add:
        facilities_copy.loc[idx, col] = feature_row[col]
      if row['shelter'] == 1:
        all_features.at[all_feat_idx, 'known_shelter_count'] += 1
      break

In [ ]:
all_features['known_shelter_count'].sum()

97

In [ ]:
all_features['shelter_count'].sum()

3464

In [ ]:
all_features[['latitude', 'longitude', 'shelter_count', 'known_shelter_count']]

,latitude,longitude,shelter_count,known_shelter_count
14178,34.90000,139.8250,0,0
14179,34.90000,139.8375,0,0
14180,34.90000,139.8500,0,0
14181,34.90000,139.8625,1,0
14182,34.90833,139.8125,0,0
...,...,...,...,...
7775,36.07500,139.7875,1,0
8279,36.08333,139.7750,2,0
8280,36.08333,139.7875,2,0
8289,36.09167,139.7750,0,0


In [ ]:
from folium.plugins import HeatMap
from folium import features
import branca.colormap as cm

In [ ]:
# Calculate the difference
all_features['difference'] = all_features['shelter_count'] - all_features['known_shelter_count']

# Expand points into boxes
boxes = []
for _, row in all_features.iterrows():
    # Bottom-left corner
    lat = row['latitude']
    lon = row['longitude']
    # Top-right corner
    lat_max = lat + lat_width
    lon_max = lon + lon_height
    # Create four corners of the box
    boxes.append([lat, lon, row['difference']])
    boxes.append([lat_max, lon, row['difference']])
    boxes.append([lat, lon_max, row['difference']])
    boxes.append([lat_max, lon_max, row['difference']])

# Initialize the map centered on an average location
map_center = [all_features['latitude'].mean(), all_features['longitude'].mean()]
heatmap_map = folium.Map(location=map_center, zoom_start=6)

# Add HeatMap
heat_data = [[box[0], box[1], box[2]] for box in boxes]
HeatMap(heat_data).add_to(heatmap_map)

# heatmap_map.save("heatmap.html")
heatmap_map

In [ ]:
# Define a colormap
min_diff = all_features['difference'].min()
max_diff = all_features['difference'].max()
colormap = cm.linear.YlOrRd_09.scale(min_diff, max_diff)

# Initialize the map centered on an average location with CartoDB Positron tiles
map_center = [all_features['latitude'].mean(), all_features['longitude'].mean()]
rectangle_map = folium.Map(location=map_center, zoom_start=6, tiles='CartoDB positron')

# Add rectangles to the map
for _, row in all_features.iterrows():
    # Bottom-left corner
    lat = row['latitude']
    lon = row['longitude']
    # Top-right corner
    lat_max = lat + lat_width
    lon_max = lon + lon_height
    # Rectangle color based on difference
    color = colormap(row['difference'])
    # Create rectangle
    folium.Rectangle(
        bounds=[[lat, lon], [lat_max, lon_max]],
        color=color,
        fill=True,
        fill_opacity=0.6,
        weight=0  # No border
    ).add_to(rectangle_map)

# Add colormap to the map
colormap.caption = 'Difference between shelter_count and known_shelter_count'
colormap.add_to(rectangle_map)

# Save or display the map
# rectangle_map.save("rectangle_map.html")
rectangle_map

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
facilities_copy

,index,admin_code,type_general,type_specific,name,address,maintenance_code,geometry,latitude,longitude,...,seashore,sea,golf_course,unknown,dist_to_water,0-14,15-64,65+,75+,80+
0,0,12101,3,03001,千葉県立美術館,中央港1-10-1,2,None,35.601220,140.101146,...,0.0,0.05,0.0,0.0,0.0,1005.885,5588.2382,1021.8556,511.186,294.4791
1,1,12101,3,03001,千葉市美術館,中央3-10-8,3,None,35.608958,140.124638,...,0.0,0.0,0.0,0.0,923.9314,741.9919,4802.4937,1591.3228,770.997,443.3104
2,2,12204,3,03001,ふなばしアンゼルセン公園子ども美術館,金堀町525,3,None,35.758910,140.064302,...,0.0,0.0,0.0,0.0,923.962684,167.1224,924.699,418.8621,276.5997,175.061
3,3,12210,3,03001,茂原市立美術館・郷土資料館,高師1345-1,3,None,35.430138,140.280233,...,None,None,None,None,None,None,None,None,None,None
4,4,12211,3,03001,成田山書道美術館,成田640,4,None,35.787491,140.321857,...,0.0,0.0,0.0,0.0,925.039121,292.7929,1468.4918,759.1446,480.1442,279.6958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4200,6998,12423,19,19013,高根保育所,本郷6937,3,None,35.415364,140.361111,...,None,None,None,None,None,None,None,None,None,None
4201,7000,12424,19,19013,関保育所,関6722,3,None,35.445861,140.358195,...,None,None,None,None,None,None,None,None,None,None
4202,7001,12424,19,19013,南白亀保育所,牛込85,3,None,35.466552,140.384579,...,None,None,None,None,None,None,None,None,None,None
4203,7003,12426,19,19013,三島野保育所,六地蔵344-2,3,None,35.452466,140.215209,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
facilities_copy = facilities_copy.drop(['geometry'], axis=1)
facilities_copy

,index,admin_code,type_general,type_specific,name,address,maintenance_code,latitude,longitude,inside_tokyo_mainland,...,seashore,sea,golf_course,unknown,dist_to_water,0-14,15-64,65+,75+,80+
0,0,12101,3,03001,千葉県立美術館,中央港1-10-1,2,35.601220,140.101146,True,...,0.0,0.05,0.0,0.0,0.0,1005.885,5588.2382,1021.8556,511.186,294.4791
1,1,12101,3,03001,千葉市美術館,中央3-10-8,3,35.608958,140.124638,True,...,0.0,0.0,0.0,0.0,923.9314,741.9919,4802.4937,1591.3228,770.997,443.3104
2,2,12204,3,03001,ふなばしアンゼルセン公園子ども美術館,金堀町525,3,35.758910,140.064302,True,...,0.0,0.0,0.0,0.0,923.962684,167.1224,924.699,418.8621,276.5997,175.061
3,3,12210,3,03001,茂原市立美術館・郷土資料館,高師1345-1,3,35.430138,140.280233,False,...,None,None,None,None,None,None,None,None,None,None
4,4,12211,3,03001,成田山書道美術館,成田640,4,35.787491,140.321857,True,...,0.0,0.0,0.0,0.0,925.039121,292.7929,1468.4918,759.1446,480.1442,279.6958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4200,6998,12423,19,19013,高根保育所,本郷6937,3,35.415364,140.361111,False,...,None,None,None,None,None,None,None,None,None,None
4201,7000,12424,19,19013,関保育所,関6722,3,35.445861,140.358195,False,...,None,None,None,None,None,None,None,None,None,None
4202,7001,12424,19,19013,南白亀保育所,牛込85,3,35.466552,140.384579,False,...,None,None,None,None,None,None,None,None,None,None
4203,7003,12426,19,19013,三島野保育所,六地蔵344-2,3,35.452466,140.215209,False,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
facilities_copy = facilities_copy.dropna()
facilities_copy

,index,admin_code,type_general,type_specific,name,address,maintenance_code,latitude,longitude,inside_tokyo_mainland,...,seashore,sea,golf_course,unknown,dist_to_water,0-14,15-64,65+,75+,80+
0,0,12101,3,03001,千葉県立美術館,中央港1-10-1,2,35.601220,140.101146,True,...,0.0,0.05,0.0,0.0,0.0,1005.885,5588.2382,1021.8556,511.186,294.4791
1,1,12101,3,03001,千葉市美術館,中央3-10-8,3,35.608958,140.124638,True,...,0.0,0.0,0.0,0.0,923.9314,741.9919,4802.4937,1591.3228,770.997,443.3104
2,2,12204,3,03001,ふなばしアンゼルセン公園子ども美術館,金堀町525,3,35.758910,140.064302,True,...,0.0,0.0,0.0,0.0,923.962684,167.1224,924.699,418.8621,276.5997,175.061
4,4,12211,3,03001,成田山書道美術館,成田640,4,35.787491,140.321857,True,...,0.0,0.0,0.0,0.0,925.039121,292.7929,1468.4918,759.1446,480.1442,279.6958
5,5,12212,3,03001,佐倉市立美術館,新町210,3,35.719214,140.229416,True,...,0.0,0.0,0.02,0.0,925.039586,582.7765,2935.1392,1832.3501,1147.6324,703.1809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4193,6983,12410,19,19013,横芝町立横芝第一保育所,栗山2267,3,35.648396,140.490618,True,...,0.0,0.0,0.0,0.0,1131.699035,74.4556,416.7178,283.8316,144.9583,85.0244
4194,6985,12410,19,19013,横芝町立上堺保育所,新島3510-1,3,35.620539,140.507813,True,...,0.0,0.0,0.0,0.0,1461.151732,32.9151,192.8731,136.5358,80.2842,49.4155
4195,6986,12410,19,19013,横芝町立大総保育所,寺方760,3,35.677420,140.460589,True,...,0.0,0.0,0.01,0.0,923.897542,7.7191,31.5142,57.5517,39.6366,31.1104
4196,6987,12410,19,19013,光町中央保育園,宮川5643,4,35.656111,140.503930,True,...,0.0,0.0,0.0,0.0,0.0,154.1523,712.0635,392.7201,228.5356,151.1938


In [ ]:
X_cols.extend(cols_to_add)
X_cols

['stories_above_ground',
 'stories_below_ground',
 'steel_frame',
 'reinforced_concrete',
 'lightweight',
 'block',
 'wood',
 'rice_paddy',
 'other_agriculture',
 'forest',
 'wasteland',
 'building',
 'road_transport',
 'rail_transport',
 'other_site',
 'river_wetland',
 'seashore',
 'sea',
 'golf_course',
 'unknown',
 'dist_to_water',
 '0-14',
 '15-64',
 '65+',
 '75+',
 '80+']

In [ ]:
X_df = facilities_copy[X_cols]
X = X_df.values
X.shape

(3632, 26)

In [ ]:
points = np.column_stack((facilities_copy['latitude'], facilities_copy['longitude']))

In [ ]:
points.shape

(3632, 2)

In [ ]:
tri = Delaunay(points)
indices = tri.simplices.flatten()

In [ ]:
adj_matrix = np.zeros((len(points), len(points)), dtype=int)

for i in range(0, len(indices), 3):
  simplex = indices[i:i+3]
  for i in range(3):
    for j in range(i+1, 3):
      adj_matrix[simplex[i], simplex[j]] = 1
      adj_matrix[simplex[j], simplex[i]] = 1

In [ ]:
adj_matrix.shape

(3632, 3632)

In [ ]:
y = facilities_copy['shelter'].values.reshape(-1, 1)
y.shape

(3632, 1)

In [ ]:
X.shape, adj_matrix.shape, y.shape

((3632, 26), (3632, 3632), (3632, 1))

In [ ]:
print('Number of shelters:', y.ravel().sum())

Number of shelters: 97


In [ ]:
# basepath = f'data/arrays/{PREF}_{"less" if drop_small_widths else "more"}/'
# np.save(basepath + 'x.npy', X)
# np.save(basepath + 'a.npy', adj_matrix)
# np.save(basepath + 'y.npy', y)
# facilities_copy.to_pickle(basepath + 'features_df.pkl')
# print('Saved to', basepath)

In [ ]:
facilities_copy.columns

Index(['index', 'admin_code', 'type_general', 'type_specific', 'name',
       'address', 'maintenance_code', 'latitude', 'longitude',
       'inside_tokyo_mainland', 'area', 'stories_above_ground',
       'stories_below_ground', 'road_width', 'nearest_station',
       'dist_to_station', 'steel_frame', 'reinforced_concrete', 'lightweight',
       'block', 'wood', 'shelter', 'rice_paddy', 'other_agriculture', 'forest',
       'wasteland', 'building', 'road_transport', 'rail_transport',
       'other_site', 'river_wetland', 'seashore', 'sea', 'golf_course',
       'unknown', 'dist_to_water', '0-14', '15-64', '65+', '75+', '80+'],
      dtype='object')

In [ ]:
X_df

,stories_above_ground,stories_below_ground,steel_frame,reinforced_concrete,lightweight,block,wood,rice_paddy,other_agriculture,forest,...,seashore,sea,golf_course,unknown,dist_to_water,0-14,15-64,65+,75+,80+
0,2,0,0,0,0,0,1,0.0,0.0,0.03,...,0.0,0.05,0.0,0.0,0.0,1005.885,5588.2382,1021.8556,511.186,294.4791
1,2,0,0,0,0,0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,923.9314,741.9919,4802.4937,1591.3228,770.997,443.3104
2,2,0,0,0,0,0,1,0.0,0.25,0.17,...,0.0,0.0,0.0,0.0,923.962684,167.1224,924.699,418.8621,276.5997,175.061
4,2,0,0,0,0,0,1,0.01,0.0,0.1,...,0.0,0.0,0.0,0.0,925.039121,292.7929,1468.4918,759.1446,480.1442,279.6958
5,2,0,0,0,0,0,1,0.02,0.0,0.13,...,0.0,0.0,0.02,0.0,925.039586,582.7765,2935.1392,1832.3501,1147.6324,703.1809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4193,2,0,0,0,0,0,1,0.61,0.16,0.04,...,0.0,0.0,0.0,0.0,1131.699035,74.4556,416.7178,283.8316,144.9583,85.0244
4194,2,0,0,0,0,0,1,0.73,0.19,0.0,...,0.0,0.0,0.0,0.0,1461.151732,32.9151,192.8731,136.5358,80.2842,49.4155
4195,2,0,0,0,0,0,1,0.17,0.2,0.5,...,0.0,0.0,0.01,0.0,923.897542,7.7191,31.5142,57.5517,39.6366,31.1104
4196,2,0,0,0,0,0,1,0.13,0.05,0.05,...,0.0,0.0,0.0,0.0,0.0,154.1523,712.0635,392.7201,228.5356,151.1938


In [ ]:
df1 = mainland_known_facilities.drop(unconnected_indices)
df1 = df1.drop(['geometry'], axis=1).dropna()
df1

,index,admin_code,type_general,type_specific,name,address,maintenance_code,latitude,longitude,inside_tokyo_mainland,...,stories_below_ground,road_width,nearest_station,dist_to_station,steel_frame,reinforced_concrete,lightweight,block,wood,shelter
0,0,12101,3,03001,千葉県立美術館,中央港1-10-1,2,35.601220,140.101146,True,...,0,5.0,千葉みなと,1200,0,0,0,0,1,0
1,1,12101,3,03001,千葉市美術館,中央3-10-8,3,35.608958,140.124638,True,...,0,4.0,千葉,550,0,0,0,0,1,0
2,2,12204,3,03001,ふなばしアンゼルセン公園子ども美術館,金堀町525,3,35.758910,140.064302,True,...,0,5.5,三咲,3200,0,0,0,0,1,0
3,3,12210,3,03001,茂原市立美術館・郷土資料館,高師1345-1,3,35.430138,140.280233,False,...,0,4.0,新茂原,1200,1,0,1,0,0,0
4,4,12211,3,03001,成田山書道美術館,成田640,4,35.787491,140.321857,True,...,0,4.5,京成成田,1000,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4200,6998,12423,19,19013,高根保育所,本郷6937,3,35.415364,140.361111,False,...,0,5.0,上総一ノ宮,2300,0,0,0,0,1,1
4201,7000,12424,19,19013,関保育所,関6722,3,35.445861,140.358195,False,...,0,4.0,八積,9100,0,0,0,0,1,0
4202,7001,12424,19,19013,南白亀保育所,牛込85,3,35.466552,140.384579,False,...,0,4.0,八積,9100,0,0,0,0,1,0
4203,7003,12426,19,19013,三島野保育所,六地蔵344-2,3,35.452466,140.215209,False,...,0,5.0,茂原,7000,0,0,0,0,1,0


In [ ]:
shelters_to_vis = df1.loc[X_df.index][['latitude', 'longitude', 'shelter']]
shelters_to_vis

,latitude,longitude,shelter
0,35.601220,140.101146,0
1,35.608958,140.124638,0
2,35.758910,140.064302,0
4,35.787491,140.321857,0
5,35.719214,140.229416,0
...,...,...,...
4193,35.648396,140.490618,1
4194,35.620539,140.507813,1
4195,35.677420,140.460589,1
4196,35.656111,140.503930,0


In [ ]:
mymap = folium.Map(location=[shelters_to_vis['latitude'].mean(), shelters_to_vis['longitude'].mean()], zoom_start=4, tiles='cartodbpositron')

# Iterate through each row of the DataFrame
for index, row in shelters_to_vis.iterrows():
    # Determine the color and opacity based on the value of the 'shelter' column
    color = '#3B3B98' if row['shelter'] == 1 else '#F97F51'
    opacity = 0.8 if row['shelter'] == 1 else 0.2

    # Add a circle marker to the map
    folium.Circle(
        location=[row['latitude'], row['longitude']],
        radius=500,  # Set radius to 500 meters
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=opacity,
        stroke=False
    ).add_to(mymap)

mymap

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
facilities_copy['shelter'].sum()

97

In [ ]:
basepath = f'data/arrays/{PREF}_{"less" if drop_small_widths else "more"}/'
facilities_copy.to_pickle(basepath + 'X_df.pkl')
df1.to_pickle(basepath + 'df1.pkl')
print('Saved to', basepath)

Saved to data/arrays/chiba_more/


In [ ]:
facilities_copy

,index,admin_code,type_general,type_specific,name,address,maintenance_code,latitude,longitude,inside_tokyo_mainland,...,seashore,sea,golf_course,unknown,dist_to_water,0-14,15-64,65+,75+,80+
0,0,12101,3,03001,千葉県立美術館,中央港1-10-1,2,35.601220,140.101146,True,...,0.0,0.05,0.0,0.0,0.0,1005.885,5588.2382,1021.8556,511.186,294.4791
1,1,12101,3,03001,千葉市美術館,中央3-10-8,3,35.608958,140.124638,True,...,0.0,0.0,0.0,0.0,923.9314,741.9919,4802.4937,1591.3228,770.997,443.3104
2,2,12204,3,03001,ふなばしアンゼルセン公園子ども美術館,金堀町525,3,35.758910,140.064302,True,...,0.0,0.0,0.0,0.0,923.962684,167.1224,924.699,418.8621,276.5997,175.061
4,4,12211,3,03001,成田山書道美術館,成田640,4,35.787491,140.321857,True,...,0.0,0.0,0.0,0.0,925.039121,292.7929,1468.4918,759.1446,480.1442,279.6958
5,5,12212,3,03001,佐倉市立美術館,新町210,3,35.719214,140.229416,True,...,0.0,0.0,0.02,0.0,925.039586,582.7765,2935.1392,1832.3501,1147.6324,703.1809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4193,6983,12410,19,19013,横芝町立横芝第一保育所,栗山2267,3,35.648396,140.490618,True,...,0.0,0.0,0.0,0.0,1131.699035,74.4556,416.7178,283.8316,144.9583,85.0244
4194,6985,12410,19,19013,横芝町立上堺保育所,新島3510-1,3,35.620539,140.507813,True,...,0.0,0.0,0.0,0.0,1461.151732,32.9151,192.8731,136.5358,80.2842,49.4155
4195,6986,12410,19,19013,横芝町立大総保育所,寺方760,3,35.677420,140.460589,True,...,0.0,0.0,0.01,0.0,923.897542,7.7191,31.5142,57.5517,39.6366,31.1104
4196,6987,12410,19,19013,光町中央保育園,宮川5643,4,35.656111,140.503930,True,...,0.0,0.0,0.0,0.0,0.0,154.1523,712.0635,392.7201,228.5356,151.1938


In [ ]:
X_df.shape

(3632, 26)

In [ ]:
df1.shape

(4011, 22)